In [1]:
import pandas as pd
import pixiedust
import numpy as np
import random
import requests
import json

Pixiedust database opened successfully


In [2]:
df = pd.read_csv('DB_ready.csv')

In [3]:
config = dict(
    username="2660e8fa-aac2-45ef-81ba-217967728ad2-bluemix", 
    password="9cd1b00e6b6927e663bfa0e2e9d677e163378a7c98e09bfb598dd74a4439fc7a",
    database="cfc_patient_db")
params = {'include_docs': 'true'}

In [4]:
url = "https://{username}:{password}@{username}.cloudant.com/{database}/_all_docs".format(**config)

In [5]:
auth_headers = {"Content-Type": "application/x-www-form-urlencoded"}
r = requests.get(url=url, params=params)
data = json.loads(r.text)

In [6]:
print(json.dumps(data, indent=2))

{
  "total_rows": 1117,
  "offset": 0,
  "rows": [
    {
      "id": "6cff7b8741389be06cb267123cf49a8a",
      "key": "6cff7b8741389be06cb267123cf49a8a",
      "value": {
        "rev": "1-1144220d44d4024b5c1b92b00252392b"
      },
      "doc": {
        "_id": "6cff7b8741389be06cb267123cf49a8a",
        "_rev": "1-1144220d44d4024b5c1b92b00252392b",
        "": "0",
        "Items": "Noodles",
        "Origin": "",
        "Items grouped": "Noodles",
        "Category": "Food - Dry Food",
        "Unit": "box",
        "amount": "5.0",
        "District": "Bhaktapur",
        "VDC": "Nankhel",
        "Village/Ward/extra info": "Maligaun",
        "lat": "27.6422",
        "lon": "85.4598",
        "District Latitude": "27.667340000000003",
        "District Longitude": "85.41673",
        "Date": "2015-04-29",
        "Location name": "Nankhel, Maligaun",
        "type": "list"
      }
    },
    {
      "id": "6cff7b8741389be06cb267123cf4a2fa",
      "key": "6cff7b8741389be06cb267123

In [7]:
df = [row["doc"] for row in data["rows"]]

In [8]:
df =pd.DataFrame(df)

In [9]:
df[["lat","lon","amount"]] = df[["lat","lon","amount"]].apply(pd.to_numeric)

In [10]:
#df[["amount"]] = df[["amount"]].astype(int)

In [11]:
display(df)

In [12]:
people = pd.DataFrame(columns=['name','needed_item','amount','location','unit'])


In [13]:
with open('people.txt','r') as f:
    names = f.readlines()

In [14]:
names = [n.strip('\n') for n in names]
people['name']= names

In [15]:
items_list = set([(doc["Items"].lower(), doc["Unit"]) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc])
locations = [(doc["Location name"], doc["lat"], doc["lon"]) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc]

In [16]:
location_dict = {' '.join(location[:-2]): (location[-2],location[-1]) for location in locations}

In [17]:
unit_lookup = {name:unit for name, unit in items_list}

In [18]:
people['needed_item'] = [random.choice(list(items_list))[0] for i in range(len(people))]

In [19]:
people['unit']=[unit_lookup[item] for item in people['needed_item']]

In [20]:
items_list

{('acenac (muscular pains)', 'tabs'),
 ('adhesive tape', 'rolls'),
 ('azifast (amoxil)', 'tabs'),
 ('bandage', 'unit'),
 ('beaten rice', 'sack'),
 ('betadine', 'bottle'),
 ('biscuits', 'box'),
 ('blankets', 'unit'),
 ('brucet', 'tabs'),
 ('brufen', 'bottle'),
 ('brufen', 'tabs'),
 ('children milk', 'pack'),
 ('colour pencils', 'pack'),
 ('colour pencils', 'unit'),
 ('cotton', 'pack'),
 ('crepe bandage', 'rolls'),
 ('dal', 'pack'),
 ('dal', 'sack'),
 ('dalmot', 'pack'),
 ('dalmot', 'sack'),
 ('decold', 'tabs'),
 ('dettol antiseptic liquids', 'bottle'),
 ('dettol soaps', 'unit'),
 ('diapers', 'box'),
 ('diapers', 'unit'),
 ('diclofenac gel', 'unit'),
 ('electrobin', 'unit'),
 ('electrolytes', 'unit'),
 ('eraser', 'unit'),
 ('feeding bottles', 'unit'),
 ('flex on (painkiller', 'box'),
 ('flexon', 'unit'),
 ('foam pads', 'unit'),
 ('food', 'pack'),
 ('food packages', 'unit'),
 ('formula milk', 'box'),
 ('fruit based energy drinks', 'box'),
 ('furandanal', 'pack'),
 ('gauge bandage', 'rolls

In [21]:
locations = pd.DataFrame([(doc["Location name"], (doc["lat"],doc["lon"])) for doc in [row["doc"] for row in data["rows"]] if "Items" in doc], columns = ['name','latlon'])

In [22]:
people['amount'] = np.random.randint(5, size = len(people))+1

In [23]:
people['location'] = [random.choice(list(locations['name'])) for i in range(len(people))]

In [24]:
station_supplies = pd.DataFrame(columns=['name','item_name','item_quanity','lat','lon'])

In [25]:
station_supplies = {}

for person in people.iterrows():
    person = person[1]
    if (person['needed_item'],person['location']) in station_supplies.keys():
        station_supplies[(person['needed_item'],person['location'])] += person['amount']
    else:
        station_supplies[(person['needed_item'],person['location'])] = person['amount']

In [26]:
station_supplies

{('acenac (muscular pains)', 'Lakhanpur, '): 3,
 ('seczylos', 'Aalapot, Kageshwari, Halapur'): 1,
 ('water', 'Shreenathkot, Hansapur'): 2,
 ('tent (13x20)', 'Marpak, '): 5,
 ('diapers', 'Chapagaun, Ghyampedanda'): 4,
 ('electrobin', 'Lakhanpur, '): 2,
 ('blankets', 'Baad Bhanjyang, Mabina'): 4,
 ('piyush', 'Tupche, Mahadev Khadka, ward 7'): 4,
 ('medicine - supplies', 'KartikeDeurali, ward 4'): 3,
 ('masks', 'Gairimudi, '): 4,
 ('dalmot', 'Aalapot, Kageshwari, Halapur'): 2,
 ('gloves', 'Gati, '): 2,
 ('azifast (amoxil)', 'Gerkhu, wards 3, 4, 9'): 2,
 ('adhesive tape', 'Nankhel, Maligaun'): 3,
 ('eraser', 'Bukhel, and also Choughare VDC'): 2,
 ('phenyl', 'Bungamati, '): 4,
 ('sugar', 'Nalang, Pipal Nalang'): 2,
 ('masks', 'Jalbire, '): 3,
 ('noodles purchased (75 bags of 20 units)', 'Phulasi, and Chanakha VDC'): 5,
 ('sketch book',
  'Dhuyang, Ghumtang, Dandagaun, Kolbari, Kimbegaun, Khanyabas, Devchagaun, Chapleti'): 5,
 ('fruit based energy drinks', 'Marpak, '): 4,
 ('gauze dressings'

In [27]:
new_df = []
for location_df in list(df.groupby('Location name')):
    temp_df=location_df[1][['amount','Items']]
    temp_df['Location name'] = location_df[0]
    temp_df.rename(columns={'amount':'current amount'}, inplace=True)
    new_df.append(temp_df)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/andrew/.local/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [28]:
current_amounts = pd.concat(new_df)
current_amounts = current_amounts.dropna(subset = ['Items'], how='all')
current_amounts

,current amount,Items,Location name
598,2.0,Biscuits,"Aalapot, Kageshwari, Halapur"
599,2.0,Noodles,"Aalapot, Kageshwari, Halapur"
600,1.0,Rice,"Aalapot, Kageshwari, Halapur"
601,1.0,Medical Kit,"Aalapot, Kageshwari, Halapur"
602,80.0,Medicine - rehydration,"Aalapot, Kageshwari, Halapur"
603,5.0,Piyush,"Aalapot, Kageshwari, Halapur"
604,13.0,Plastic rolls,"Aalapot, Kageshwari, Halapur"
645,1.0,Medical Kit,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
646,60.0,Medicine - rehydration,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"
647,1.0,Sanitary pads,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan"


In [29]:
current_amounts_lookup = {}
for value in current_amounts.values:
    if value[2] in current_amounts_lookup:
        current_amounts_lookup[value[2]][value[1]] = value[0]
    else:
        current_amounts_lookup[value[2]] = {value[1]:value[0]}
    


In [30]:
matrix = []

for key in station_supplies.keys():
    item = key[0]
    location = key[1]
    amount = station_supplies[key]
    if location in current_amounts_lookup:
        if item in current_amounts_lookup[location]:
            requested = (amount - current_amounts_lookup[location][item])
            print(requested,location,item)
        else:
            requested = amount
    else: requested = amount
    
    
    matrix.append(pd.Series([item, location, amount, requested, unit_lookup[item], *location_dict[location]],index=['needed_item','Location','amount needed','requested', 'unit', 'lat', 'long']))

In [31]:
final_data = pd.DataFrame(matrix)

In [43]:
final_data['amount requested'] = [int(np.random.normal(x,)) for x in final_data['amount needed']]

In [48]:
display(final_data)


In [34]:
pd.merge(current_amounts, final_data,  how='left', left_on=['Location name','Items'], right_on = ['Location','needed_item'])

,current amount,Items,Location name,needed_item,Location,amount needed,requested,unit,lat,long
0,2.0,Biscuits,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Noodles,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Rice,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Medical Kit,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,80.0,Medicine - rehydration,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,Piyush,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13.0,Plastic rolls,"Aalapot, Kageshwari, Halapur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Medical Kit,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,60.0,Medicine - rehydration,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,Sanitary pads,"Aanppipal, Bhorle, Jaljale, Sunodumre, Pachchyan",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
display(final_data)

needed_item,Location,amount needed,requested,unit,lat,long
acenac (muscular pains),"Lakhanpur,",3,3,tabs,27.563240000000004,85.86010999999999
seczylos,"Aalapot, Kageshwari, Halapur",1,1,box,27.74768,85.43146
water,"Shreenathkot, Hansapur",2,2,box,28.087,84.6393
tent (13x20),"Marpak,",5,5,unit,28.0135,84.9646
diapers,"Chapagaun, Ghyampedanda",4,4,unit,27.5935,85.3297
electrobin,"Lakhanpur,",2,2,unit,27.563240000000004,85.86010999999999
blankets,"Baad Bhanjyang, Mabina",4,4,unit,27.704759999999997,85.2104
piyush,"Tupche, Mahadev Khadka, ward 7",4,4,bottle,27.965670000000003,85.1539
medicine - supplies,"KartikeDeurali, ward 4",3,3,box,27.5472,85.802
masks,"Gairimudi,",4,4,unit,27.604290000000002,86.12272


In [47]:
final_data.to_pickle('final_data.p')